In [1]:
import pandas as pd
# https://files.grouplens.org/datasets/movielens/ml-25m.zip
movies = pd.read_csv(r"C:\Users\dipak\Downloads\ml-25m\ml-25m\movies.csv")


In [2]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


# Phase 1: Data Loading and Preprocessing

### 1. Initial Data Inspection
Loaded the movie dataset and displayed the first few entries to inspect columns like `movieId`, `title`, and `genres`.




The original titles contain years and special characters (like Toy Story (1995)). If a user searches for "Toy Story," the parentheses and numbers might interfere with the search accuracy. By removing everything except letters, numbers, and spaces, we create a "normalized" version of the title that is much easier for the computer to match.

In [3]:
import re

def clean_title(title):
    title = re.sub("[^a-zA-Z0-9 ]", "", title)
    return title

### I have defined a function called clean_title using Regular Expressions (regex). Now, I am applying this function to the title column to create a new column called clean_title.

In [4]:
movies["clean_title"] = movies["title"].apply(clean_title)

In [5]:
movies

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
62418,209157,We (2018),Drama,We 2018
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul 2001
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems 2018
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing 2001


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(movies["clean_title"])

## Text Vectorization with TF-IDF

To transform our movie titles into a format the machine can understand, we utilize the **TfidfVectorizer**. This process converts raw text into a **Sparse Matrix**, where each title is represented by a set of weighted numerical values.

### 🔍 How it Works:
* **Feature Identification**: The vectorizer scans the `clean_title` column to identify all unique words (**unigrams**) and pairs of consecutive words (**bigrams**) across the entire dataset.
* **Statistical Weighting**: Each term is assigned a weight based on its frequency:
    * **Term Frequency (TF)**: How often a word appears in a specific title.
    * **Inverse Document Frequency (IDF)**: how unique that word is across the whole dataset (penalizing common words like "The" or "Movie").



> ### 💡 Why use `ngram_range=(1,2)`?
> This setting is the "secret sauce" for title matching. It allows the system to recognize **"Toy"** and **"Story"** as individual concepts, but crucially, it also identifies **"Toy Story"** as a single, unique entity. This ensures much higher accuracy when searching for multi-word movie titles.

##  Measuring Similarity with Cosine Similarity

With our titles converted into mathematical vectors, we now need a way to calculate how "close" a user's search query is to the movies in our database. We achieve this using **Cosine Similarity**.

### 🧠 The Logic Behind the Search
Instead of looking at word counts, Cosine Similarity calculates the **angle between two vectors**. 
* **Score of 1.0**: The vectors point in the exact same direction (identical titles).
* **Score of 0.0**: The vectors are orthogonal (no shared words).



**Our search function will execute a four-step pipeline:**
1. **Normalization**: Clean the input search term using our `clean_title` function.
2. **Vectorization**: Transform the search term into the TF-IDF space.
3. **Comparison**: Calculate the similarity score between the query vector and every movie in our dataset.
4. **Ranking**: Extract the top 5 most relevant matches.

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies.iloc[indices].iloc[::-1]
    
    return results

### ⚡ Implementation Highlights
The `search` function is optimized for speed and accuracy through two key programming choices:

* **`similarity.flatten()`**: We convert the similarity matrix into a simple 1D array to make it easier to sort and index.
* **`np.argpartition`**: Unlike a full sort (which is computationally expensive), `argpartition` quickly finds the indices of the **top 5** highest scores. 
* **`iloc[::-1]`**: Since NumPy's partition doesn't guarantee a specific order within the top 5, we reverse the results to ensure the **most relevant** match appears at the top of the list.

**The system is now ready to transform any text input into a ranked list of movie matches!**

In [8]:
!pip install ipywidgets


In [9]:
import ipywidgets as widgets
widgets.IntSlider(value=50, description='Test:')

IntSlider(value=50, description='Test:')

In [10]:
%pip install --upgrade ipywidgets jupyterlab_widgets

Note: you may need to restart the kernel to use updated packages.


In [11]:
import ipywidgets as widgets
widgets.IntSlider(value=50, description='Test:')

IntSlider(value=50, description='Test:')

In [12]:
# 1. Force upgrade the packages
%pip install --upgrade ipywidgets jupyterlab_widgets

# 2. Check that the versions are now ipywidgets 8.x
import ipywidgets
print(f"New version: {ipywidgets.__version__}")

Note: you may need to restart the kernel to use updated packages.
New version: 8.1.8


## Creating an Interactive Search Widget

To make our recommendation system more practical, we will build an interactive search interface using `ipywidgets`. This allows a user to type a movie title directly into a search box and see matches instantly without re-running code cells manually.

**Technical Strategy:**
* **`widgets.Text`**: Creates a text entry field for the user's query.
* **`widgets.Output`**: Acts as a dedicated container for the search results, preventing the notebook from becoming cluttered.
* **Event Observation**: We use the `.observe` method to watch for changes in the text box.
* **Efficiency Logic**: The search function only triggers when the input exceeds 5 characters, reducing unnecessary computation while the user is still typing.

In [13]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# 1. Create the text input
movie_input = widgets.Text(
    value='Toy Story 1995',
    description='Movie Title:',
    disabled=False
)

# 2. Create the area where results will be shown
movie_list = widgets.Output()

def on_type(data):
    # This block ensures everything happens inside the 'movie_list' area
    with movie_list:
        # Clear previous results immediately
        clear_output(wait=True)
        
        title = data["new"]
        
        # We search if the title is longer than 5 chars
        if len(title) > 5:
            try:
                results = search(title)
                if not results.empty:
                    print(f"Showing results for: {title}")
                    display(results[['movieId', 'title', 'genres']])
                else:
                    print("No matches found.")
            except Exception as e:
                print(f"Error during search: {e}")

# 3. Link the text box to the function
movie_input.observe(on_type, names='value')

# 4. Display BOTH the input and the list area
display(movie_input)
display(movie_list)

Text(value='Toy Story 1995', description='Movie Title:')

Output()

In [14]:
# Manual test
test_result = search("Toy Story 1995 ")
print(test_result)

       movieId                       title  \
0            1            Toy Story (1995)   
3021      3114          Toy Story 2 (1999)   
59767   201588          Toy Story 4 (2019)   
14813    78499          Toy Story 3 (2010)   
20497   106022  Toy Story of Terror (2013)   

                                                 genres  \
0           Adventure|Animation|Children|Comedy|Fantasy   
3021        Adventure|Animation|Children|Comedy|Fantasy   
59767               Adventure|Animation|Children|Comedy   
14813  Adventure|Animation|Children|Comedy|Fantasy|IMAX   
20497                         Animation|Children|Comedy   

                    clean_title  
0                Toy Story 1995  
3021           Toy Story 2 1999  
59767          Toy Story 4 2019  
14813          Toy Story 3 2010  
20497  Toy Story of Terror 2013  


## Creating an Interactive Search Widget

To make our recommendation system more practical, we will build an interactive search interface using `ipywidgets`. This allows a user to type a movie title directly into a search box and see matches instantly without re-running code cells manually.

**Technical Strategy:**
* **`widgets.Text`**: Creates a text entry field for the user's query.
* **`widgets.Output`**: Acts as a dedicated container for the search results, preventing the notebook from becoming cluttered.
* **Event Observation**: We use the `.observe` method to watch for changes in the text box.
* **Efficiency Logic**: The search function only triggers when the input exceeds 5 characters, reducing unnecessary computation while the user is still typing.ction!**

## Finding Similar Users (Collaborative Filtering)

Now that we can search for movies by title, the next goal is to recommend movies based on user behavior. This is known as **Collaborative Filtering**. Our logic follows the principle: *"If a user liked the movie we searched for, what other movies did they also enjoy?"*

**In this section, we will:**
1. **Identify a Target Movie**: Select a specific `movieId` (e.g., Avengers: Age of Ultron) to act as our recommendation anchor.
2. **Load User Ratings**: Import the `ratings.csv` dataset containing millions of user interactions.
3. **Filter for "High-Value" Users**: Find users who not only watched our target movie but also gave it a high rating (greater than 4 stars). These users represent our "similarity group."

In [15]:
movie_id = 89745

#def find_similar_movies(movie_id):
movie = movies[movies["movieId"] == movie_id]

I am selecting a specific movieId (in this case, 89745 for The Avengers) and retrieving its record from my dataset.

In [16]:
ratings = pd.read_csv(r"C:\Users\dipak\Downloads\ml-25m\ml-25m\ratings.csv")


In [17]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

The core logic of a recommendation engine is: "If User A and User B both liked Movie X, then User A might also like other movies that User B enjoyed." By narrowing our list to users who rated this specific movie highly, we are identifying a group of people whose preferences are relevant to our target. Using .unique() ensures we don't count the same user multiple times if there were duplicate entries.

In [18]:
similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()

I am filtering the ratings dataset to find the unique IDs of users who watched the same movie (movie_id = 89745) and gave it a high rating (greater than 4 stars).

In [19]:
similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]

### Identifying the "Similar User" Pool
By executing the code above, we have successfully isolated the `userIds` of people who share an interest in our target movie. 

* **The `similar_users` Variable**: This now contains a unique list of users who are "fans" of the selected `movieId`. 
* **Data Selection**: By filtering for ratings `> 4`, we ensure that our recommendations will be based on positive experiences rather than just any user who happened to watch the film.



**The next step will be to find all other movies these specific users liked, which will help us identify patterns and popular films within this specific sub-community of movie fans.**

## Calculating Recommendation Scores

Now that we have identified users who liked our target movie, we need to determine which other movies they recommend. However, simply picking the most popular movies among these users isn't enough—standard blockbusters (like *The Avengers* or *Forrest Gump*) tend to be popular with everyone.

To find **niche recommendations**, we will calculate a ratio between:
1.  **Similar Percentage**: How many "similar users" liked a movie.
2.  **All Percentage**: How many "average users" in the entire dataset liked that same movie.

**The Strategy:**
* **Filter Frequency**: We only look at movies that at least 10% of similar users liked.
* **The Score**: We divide the "similar" percentage by the "all" percentage. A high score means the movie is much more popular among fans of our target film than it is with the general public.

In [20]:
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

similar_user_recs = similar_user_recs[similar_user_recs > .10]

In [21]:
all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]

In [22]:
all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

In [23]:
rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
rec_percentages.columns = ["similar", "all"]

In [24]:
rec_percentages

,similar,all
movieId,,
89745,1.000000,0.040459
58559,0.573393,0.148256
59315,0.530649,0.054931
79132,0.519715,0.132987
2571,0.496687,0.247010
...,...,...
47610,0.103545,0.022770
780,0.103380,0.054723
88744,0.103048,0.010383


In [25]:
rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]

In [26]:
rec_percentages = rec_percentages.sort_values("score", ascending=False)

In [27]:
rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")

,similar,all,score,movieId,title,genres,clean_title
17067,1.000000,0.040459,24.716368,89745,"Avengers, The (2012)",Action|Adventure|Sci-Fi|IMAX,Avengers The 2012
20513,0.103711,0.005289,19.610199,106072,Thor: The Dark World (2013),Action|Adventure|Fantasy|IMAX,Thor The Dark World 2013
25058,0.241054,0.012367,19.491770,122892,Avengers: Age of Ultron (2015),Action|Adventure|Sci-Fi,Avengers Age of Ultron 2015
19678,0.216534,0.012119,17.867419,102125,Iron Man 3 (2013),Action|Sci-Fi|Thriller|IMAX,Iron Man 3 2013
16725,0.215043,0.012052,17.843074,88140,Captain America: The First Avenger (2011),Action|Adventure|Sci-Fi|Thriller|War,Captain America The First Avenger 2011
16312,0.175447,0.010142,17.299824,86332,Thor (2011),Action|Adventure|Drama|Fantasy|IMAX,Thor 2011
21348,0.287608,0.016737,17.183667,110102,Captain America: The Winter Soldier (2014),Action|Adventure|Sci-Fi|IMAX,Captain America The Winter Soldier 2014
25071,0.214049,0.012856,16.649399,122920,Captain America: Civil War (2016),Action|Sci-Fi|Thriller,Captain America Civil War 2016
25061,0.136017,0.008573,15.865628,122900,Ant-Man (2015),Action|Adventure|Sci-Fi,AntMan 2015
14628,0.242876,0.015517,15.651921,77561,Iron Man 2 (2010),Action|Adventure|Sci-Fi|Thriller|IMAX,Iron Man 2 2010


### Final Recommendation Results
By merging our scores with the `movies` metadata, we now have our final list of recommendations. 

**Key Metrics Explained:**
* **`similar`**: The percentage of users who liked our target movie and also liked this movie.
* **`all`**: The percentage of the general population who liked this movie.
* **`score`**: The strength of the recommendation. For example, if a movie has a score of `5.0`, it means users similar to us are **5 times more likely** to enjoy this movie than the average person.

This method effectively filters out "generic" popularity and highlights movies that share a genuine thematic or stylistic connection with our search term.

## Finalizing the Recommendation Engine

To make our system scalable and easy to use, we are wrapping the entire logic—from finding similar users to calculating the recommendation scores—into a single function called `find_similar_movies`. 

**This function executes the following workflow:**
1.  **Audience Identification**: Finds users who gave the target movie a rating of 4 or higher.
2.  **Preference Mapping**: Collects all other movies that this specific group of users also highly rated.
3.  **Statistical Filtering**: Keeps only the movies liked by more than 10% of that audience to ensure meaningful patterns.
4.  **Relative Popularity Calculation**: Compares how much "similar users" liked a movie versus the "entire dataset."
5.  **Scoring & Sorting**: Ranks movies by the highest ratio (the "score") and merges them with our movie metadata for a readable final list.

In [28]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]

### Summary of the Function Output
The `find_similar_movies` function is now the core engine of our project. By returning the `score`, `title`, and `genres`, it provides a clear answer to why a movie is being recommended.

* **Higher Scores** indicate "hidden gems"—movies that are specifically beloved by fans of your chosen title but might not be mainstream hits.
* **Lower Scores (closer to 1)** would indicate movies that are popular generally, but not necessarily unique to this specific fan base.

**The final step is to integrate this function with our interactive search widget to create a seamless, end-to-end movie discovery experience!**

## Creating the Final Interactive Application

We have now reached the final stage of our project: integrating the **Search Engine** with the **Recommendation Engine**. This creates a seamless, real-time experience where typing a movie title immediately generates a list of personalized recommendations.

**How this final integration works:**
1. **Dynamic Search**: As you type, the `search()` function identifies the most likely movie you are looking for.
2. **Automatic Selection**: The system automatically picks the top result (`iloc[0]`) from the search results and extracts its `movieId`.
3. **Recommendation Trigger**: That `movieId` is passed instantly into our `find_similar_movies()` function.
4. **Real-Time Display**: Using the `widgets.Output` container, the final list of recommended movies is displayed and updated with every keystroke.

In [29]:
import ipywidgets as widgets
from IPython.display import display

movie_name_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()

### Project Conclusion: A Complete Recommendation System
I have successfully built a sophisticated movie recommendation engine from scratch using Python. 

**Summary of the Technical Achievement:**
* **Text Analysis**: Used TF-IDF and N-Grams to understand movie titles.
* **Similarity Modeling**: Implemented Cosine Similarity for fast search retrieval.
* **Collaborative Filtering**: Analyzed millions of user ratings to find common preferences.
* **Scoring Algorithm**: Created a custom "Recommendation Score" to prioritize niche interests over generic popularity.
* **Interactive UI**: Leveraged Jupyter Widgets to turn data science code into a functional tool.

